# 트리의 앙상블

정형데이터
- CSV 같이 어떤 구조로 정리되어 있는 데이터 
- EX_ 엑셀 , csv , 데이터 베이스 등 

비정형 데이터 
- 엑셀이나 데이터베이스로 표현하기 어려운것
- 텍스트 데이터 , 사진 , 음악 

앙상블 학습 
- 정형 데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘 
- 대부분 결정 트리 기반으로 만들어짐 


신경망 알고리즘
- 비정형 데이터 다룰 때 사용
- 규칙성을 찾기 어려울 때 



## 랜덤포레스트

- 앙상블 학습의 대표 주자 
- 결정 트리를 랜덤하게 만들어 결정 트리 (나무)들의 숲으로 만들어 두고 각 결정 트리의 예측을 사용해서 최종 사용할 트리를 만든다 

### 트리 훈령용 데이터 만들기
- 훈련 데이터에서 랜덤하게 데이터를 추출하는데 중복이 가능 
- 즉 훈련에 사용한 데이터를 쓰고 다시 또 뽑힐 가능성이 있다. 
- 제비뽑기가 아님 
- 이를 **부트스트램 샘플** 이라함 
- 부트스트랩 : 데이터 세으테엇 중복을 허용해 데이터 샘플링 하는 방식 




In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

# 트레이닝셋 테스트셋 만들기 
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

#cross_balidate() 함수 사용 교차검증 
# n_jobs= -1 cpu 최대 사용 
# return_train_score=True : 검증 점수 및 훈렌세트 점수 반환 
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 훈련세트가 99퍼니까 좀 과대적합의 경향이 있다 

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)
# 특성 중요도 파악 
# 앞에 예제 5.2와 비교하면 당도 중요성이 감소하고 알코올 도수와 ph 중요도가 상승 
# 랜덤 포레스트는 특성 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문임 -> 하나의 특성에 과도하게 집중하는 것이 아니라
# 더 많은 특성이 훈련에 기여함 
# 과대 적합 줄이고 일반화 성능 늘림 

[0.23167441 0.50039841 0.26792718]


In [ ]:
# 부트 스트랩은 랜덤으로 중복을 허용해서 샘플을 뽑는데 그러다 보면 안 뽑히는 샘플들이 생김-> OBB
# 0BB (out of bag) : 남는 샘플 // 이를 활용하여 부트스트랩 샘플로 훈련한 결정 트리 평가 가능 마루데 검증 세트 역할 
# oob_score=True 를 통해 각 결정 트리 oob 점수 평균 출력 
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리

- 부트스트랩 지원 x : 중복 허용 x  -> 결정 트리 만들 때 전체 훈련세트 사용 
- 노드 분할 시 가장좋은 분할을 찾는 것이아니라 무작위로 분할 
- 특성을 무작위로 분할하기에 성능은 낮아지지만 많은 트리를 앙성블 하기에 과대적합을 막고 검증세트 점수를 높임 

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

- 깊이가 얕은 결정 트리 사용하여 이전 트리의 오차를 보완하는 방식의 앙상블
- 기본 깊이 3인 결정트리 100개 사용 -> 과대적합에 강하고 높은 일반화성능 
- 경사하강법을 통해 트리를 앙상블에 추가  : 경사가 낮은 곳으로 조금씩 이동 그레이디언트 붑스팅도 깊이가 얕은 트리로 찾음 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 부스팅

- 인기 있는 정형 데이터용ㅇ 알고리즘 

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
hgb.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8827690284750664 0.8708899089361072


#### LightGBM

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
